In [1]:
import numpy as np
import pandas as pd
import sympy as sp

from utilities import *

In [2]:
SLOPE_ANGLE = 3 * sp.pi / 360
DETECTOR_ANGLE = 2 * sp.pi / 3
CENTER_WATER_DEPTH = 110

PRECISION = 20

NM_TO_M = 1852
LENGTH = 4 * NM_TO_M
WIDTH = 2 * NM_TO_M

In [3]:
field = Field(length=LENGTH, width=WIDTH, center_depth=CENTER_WATER_DEPTH, slope_angle=SLOPE_ANGLE, precision=PRECISION)

In [7]:
field.arrange_measuring_lines(
    first_left_measuring_line_from_top = 20, 
    direction_angle =  89 * sp.pi / 180, 
    detector_angle = DETECTOR_ANGLE
)

finished calc first_left_measuring_line
finished calc first_top_measuring_line
finished calc top_measuring_lines: 2 in total
overlap_ratio_3: 0.22172746515394435469
left: 2 in total


<ArrangeError.OVERLAP_RATIO_TOO_LARGE: 2>